# Analyzing the composition of the mantle zone

The germinal centers were easily identified in neighborhood analysis, but, their mantle zones seemed distinct among disease tpyes

approach:
1. cluster germinal center cells based on their known labels
    - use convex hull approach
2. expand the encompassing polygon by an arbitrary distance
3. make a new donut shape
4. count relative cell abundance within the "donut"/mantle

ALSO TRY CONTOUR APPROACH

--CD--
Motif 2 is follicle, in MCD, Motif 4 is also follicle


In [1]:
import tifffile as tff
import re
# from cv2 import imwrite
import pandas as pd
import numpy as np
import pickle as pk
import matplotlib.pyplot as plt
import matplotlib as mplb
import matplotlib.cm as cm
from tqdm import tqdm
from imantics import Polygons, Mask
import geojson as gjson
import argparse
from scipy import signal, ndimage
import scipy as sci
import random
import os
import anndata as ad
from PIL import Image

from scipy.spatial import ConvexHull, convex_hull_plot_2d
from skimage.draw import polygon2mask
import shapely as shp
import geopandas as gpd

import seaborn as sns
import matplotlib as mpl

ERROR 1: PROJ: proj_create_from_database: Open of /home/smithd30/miniconda3/envs/imageAnalysis/share/proj failed


In [2]:
codex = ad.read_h5ad("/mnt/isilon/cscb/Projects/codex/pillaiv/SCTC-VP-15/code/CD_analysis/aSMA_CD21_analysis/all_regions_dat.h5ad")
codex

AnnData object with n_obs × n_vars = 3895114 × 38
    obs: 'Unnamed: 0', 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'Reg', 'Tile.Row', 'Tile.Col', 'Tile.Z', 'In.Tile.Y', 'In.Tile.X', 'Absolute.Y', 'Absolute.X', 'Cell.Size', 'n_genes_by_counts', 'total_counts', 'nCount_raw', 'nFeature_raw', 'cell_type_pred_knn', 'cell_type_pred_knn_prob', 'motif', 'index_info', 'new_pheno', 'region'
    uns: 'lda_10', 'lda_10_probability', 'lda_12', 'lda_12_probability', 'lda_15', 'lda_15_probability', 'lda_8', 'lda_8_probability', 'spatial_lda', 'spatial_lda_probability'

In [3]:
codex.obs.motif.unique()

['Motif_3', 'Motif_8', 'Motif_9', 'Motif_7', 'Motif_1', ..., 'Motif_6', 'Motif_5', 'Motif_2', 'Motif_4', 'Motif_11']
Length: 12
Categories (12, object): ['Motif_0', 'Motif_1', 'Motif_2', 'Motif_3', ..., 'Motif_8', 'Motif_9', 'Motif_10', 'Motif_11']

# Running all data

In [137]:
codex.obs['follicle'] = 0
codex.obs['follicle_level'] = -1
codex.obs['follicle_area'] = 0
for full_r in tqdm(codex.obs.region.unique()):
    print(full_r)
    adata = codex[codex.obs.region == full_r,:]
    all_coords = adata.obs[['Absolute.Y', 'Absolute.X']].to_numpy()
    all_coords = gpd.points_from_xy(x=all_coords[:,1], y=all_coords[:,0])
    r_ = full_r.split("_")
    sname = r_[0]
    regname = r_[1]

    # getting the data
    coords = adata.obs[['Absolute.Y', 'Absolute.X']].to_numpy() # should it be X,Y or Y,X ????
    coords = coords[adata.obs.motif.isin(['Motif_2', 'Motif_4']),:]

    # getting the follicles
    clustered = []
    follicle_sets = []
    for i in range(coords.shape[0]):
        if i not in clustered:
            i_dist = np.linalg.norm(coords[i]-coords, axis=1)
            i_neighbor = np.where(i_dist < 30)[0].tolist()
            # should we check whether any of the neighbors are in the clustered list?
            # i_neighbor = i_neighbor[~list(map(lambda i_: i_ in clustered, i_neighbor))]
            clustered.extend(i_neighbor)
            
            new_cl_token = 1
            fs_hits = [fi for fi in range(len(follicle_sets)) if len(set(follicle_sets[fi]).intersection(i_neighbor)) > 0]
            if len(fs_hits) > 0:
                cell_set = []
                for fi in sorted(fs_hits, reverse=True):
                    cell_set.extend(follicle_sets[fi])
                    follicle_sets.pop(fi)
                cell_set = set(cell_set).union(i_neighbor)
                follicle_sets.append(cell_set)
            else:
                follicle_sets.append(i_neighbor)


    # filtering follicles
    f_clust = []
    keep_i = []
    cl_i = 1
    for fs in follicle_sets:
        if len(fs) > 200:
            f_clust.extend(np.repeat(cl_i, len(fs)))
            keep_i.extend(fs)
            cl_i += 1
    follicle_filt = pd.DataFrame({'f_clust':f_clust,
                                 'celli':keep_i})

    # Getting the follicle data
    contour_size = 100
    fl_dfs = []
    for f in follicle_filt.f_clust.unique():
        coords_test = coords[follicle_filt.celli.loc[follicle_filt.f_clust==f].values,:]
        foll_hull = ConvexHull(coords_test)
        foll_geom = np.flip(foll_hull.points[foll_hull.vertices], 1)
        foll_poly = shp.Polygon(foll_geom)
        foll_all_poly = foll_poly.buffer(contour_size, single_sided=True)
        mantle_c = all_coords.within(foll_all_poly)
        mantle_bc = adata.obs_names[mantle_c]
        codex.obs.loc[mantle_bc, "follicle"] = f
        codex.obs.loc[mantle_bc, "follicle_area"] = foll_poly.area
        for fl in range(10):
            foll_lvl_poly = foll_poly.buffer(-contour_size*fl, single_sided=True)
            mantle = foll_lvl_poly.buffer(contour_size, single_sided=True)
            mantle_c = all_coords.within(mantle - foll_lvl_poly)
            mantle_bc = adata.obs_names[mantle_c]
            if np.sum(mantle_c) > 0:
                codex.obs.loc[mantle_bc, "follicle_level"] = 10-fl
            else:
                germ_coord = all_coords.within(foll_lvl_poly)
                germ_coord_bc = adata.obs_names[germ_coord]
                codex.obs.loc[germ_coord_bc, "follicle_level"] = 10-fl
                break
        germ_coord = all_coords.within(foll_lvl_poly)
        if np.sum(germ_coord) > 0:
            germ_coord_bc = adata.obs_names[germ_coord]
            codex.obs.loc[germ_coord_bc, "follicle_level"] = 0

  0%|          | 0/24 [00:00<?, ?it/s]

MCD1_reg1


  4%|▍         | 1/24 [01:12<27:47, 72.51s/it]

MCD3_reg2


  8%|▊         | 2/24 [01:37<16:15, 44.36s/it]

...

 92%|█████████▏| 22/24 [1:21:26<12:07, 363.87s/it]

HVCD2_reg4


 96%|█████████▌| 23/24 [1:25:53<05:34, 334.90s/it]

R1_reg1


100%|██████████| 24/24 [1:31:55<00:00, 229.81s/it]


In [5]:
# with open("mantle_df.pk","wb") as f:
#     pk.dump(all_fl_df, f)
codex.obs.to_csv("allmeta_germ_contour100.csv")